# Reading and Cleaning Data

The data I am using is of **daily PM2.5 and AQI** levels at test sites across **California** from **2010-2018**. This notebook combines all the AQI files sourced from [the EPA](https://www.epa.gov/outdoor-air-quality-data/download-daily-data). It then cleans the data by dealing sorting, dealing with NaN values, and consolidating "duplicate" rows (measurements that were at the same test site on the same day).

**Warning!** Running this notebook takes me 2-3 hours. You'll also need to download all the EPA data beforehand (see instructions in cell under "Read in Data"). No need to run this notebook if you have the pickle file already (which is included in the zip file).

---

**Getting data from online (not implemented):**
- [`requests` Scrape website requiring login (Article)](http://kazuar.github.io/scraping-tutorial/)
- [`requests` Login and download specific file w/ parameters (SO)](https://stackoverflow.com/questions/45107839/python-login-and-download-specific-file-from-website)
- [`urllib` `mechanize` Using Python to sign into website, fill in a form, then sign out (SO)](https://stackoverflow.com/questions/8560959/using-python-to-sign-into-website-fill-in-a-form-then-sign-out)
- [`mechanize` Fill online form (Blog)](https://www.thetaranights.com/fill-online-form-using-python/)
- [`urllib` `requests` `mechanize` How to submit a web form (Article)](https://www.blog.pythonlibrary.org/2012/06/08/python-101-how-to-submit-a-web-form/)
- [`webbrowser` `requests` `BeautifulSoup` `Selenium` Web Scraping (Book)](https://automatetheboringstuff.com/chapter11/)

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import time
import datetime as dt

## Read in Data

In [ ]:
# Data from 2010-2018. Change to range(2010,2020) if you have 2019 data too!
daterange = range(2010, 2019)

# Go to https://www.epa.gov/outdoor-air-quality-data/download-daily-data
# and download PM2.5 data, year, California (for all years you want).
# Filenames are 'CA[year]_dailyAQI.csv', so 'CA2018_dailyAQI.csv' for 2018.
# Store in folder: Data > dailyAQI
df_list = [pd.read_csv('Data/dailyAQI/CA'+str(i)+'_dailyAQI.csv') for i in daterange]
df_all = pd.concat(df_list, ignore_index=True)

print('Warning - this may take up to a minute.')
df_all['Date'] = pd.to_datetime(df_all['Date'])

print('Shape:', df_all.shape)
df_all.head()

In [ ]:
print('Most recent date in dataset: {}'.format(df_all.Date.max().strftime('%m/%d/%Y')))

## Clean Data

### Remove NaN

In [ ]:
def remove_nan(df):
    null_bool = df.isnull().any(axis=1) # all rows with NaN

    for i, b in enumerate(null_bool):
        if b: # NaN in this row
            county = df['COUNTY'][i]

            site, code, name = df['Site Name'][i], df['CBSA_CODE'][i], df['CBSA_NAME'][i]

            # Change NaNs to meaningful info
            if isinstance(site, float) and np.isnan(site):
                df.at[i, 'Site Name'] = county + ' (ID:' + str(df['Site ID'][i]) + ')'
            if isinstance(code, float) and np.isnan(code):
                df.at[i, 'CBSA_CODE'] = 0
            if isinstance(name, float) and np.isnan(name):
                df.at[i, 'CBSA_NAME'] = county + ', CA'
    
    return df

In [ ]:
df_all = remove_nan(df_all)
df_all = df_all.sort_values(['Site ID','Date']) # Sort by Site, date

In [ ]:
print('NaN check:')
df_all[df_all['COUNTY'] == 'Alpine'][:2]

### Fix Lebec!
Lebec has two sets of data, one in Kern county and one in Los Angeles county. I'm just updating its site name to also include the site ID (so we can distinguish them).

#### Distinguish duplicate site names (not sure how to do this more generally yet)
Some sites are in more than one county (?!). In order to distinguish them, I'm updating these site names to also include their site ID, like when I updated NaN site names.

In [ ]:
# WORK IN PROGRESS...
def update_dup_sites(df):
    sites = np.sort(df['Site Name'].unique())
    
    for site in sites:
        county_list = df[df['Site Name'] == site]['COUNTY'].unique()
        if len(county_list) > 1:
            for county in county_list:
                df_bool = (df['COUNTY'] == county) & (df['Site Name'] == site)
                ID = str(df[df_bool]['Site ID'].iloc[0])
                
                new_site = site + ' (ID:' + ID + ')'
                df.loc[df['Site Name'] == site, 'Site Name'] = new_site
    
    return df

In [ ]:
kern = 'Lebec (ID:60292009)' # new site name for Kern county's Lebec
la = 'Lebec (ID:60379034)' # new site name for LA county's Lebec
for i in range(len(df_all)):
    if df_all['Site Name'][i] == 'Lebec':
        if df_all['COUNTY'][i] == 'Kern':
            df_all.at[i, 'Site Name'] = kern
        else: # Los Angeles
            df_all.at[i, 'Site Name'] = la

In [ ]:
print('Lebec check:')
display(df_all[df_all['Site Name'] == kern][:2])
display(df_all[df_all['Site Name'] == la][:2])

In [ ]:
print('Before condensing (duplicates):')
df_all[df_all['Site Name'] == 'Los Angeles-North Main Street'][:5]

### Condense rows at same site
Some sites have more than one tool that measures AQI! That means that there’s more than one data point per date at that site. I am averging the multiple AQIs here.


In [ ]:
def condense_rows(df, div):
    start = time.time()
    
    tot_count = 0 # how many rows condensed
    row = len(df)-1 # row number. Going backwards to avoid index shift

    # for progress printing
    progress = 0
    p_mod = len(df)//div
    print('Expected final count - {}: {} (will be lower)'.format(div//10, div))

    while(True):
        count = 1 # number of rows of same site
        PM25_sum = 0 # sum of PM2.5 at site
        AQI_sum = 0 # sum AQI at site

        while(df.iloc[row,2] == df.iloc[row-1, 2] and
              df.iloc[row,0] == df.iloc[row-1, 0]): # while still same site and day...
            PM25_sum += df.iloc[row,4]
            AQI_sum += df.iloc[row,6]
            count += 1
            df = df.drop(df.iloc[row].name) # drop row

            if row > 1: row -= 1
            else: break

        if count > 1:
            # while loop did not add these to sum
            PM25_sum += df.iloc[row,4]
            AQI_sum += df.iloc[row,6]

            df.iat[row,3] = 0 # Set POC to 0, since multiple
            df.iat[row,4] = np.round(PM25_sum/count, 1) # average of PM2.5 at site
            df.iat[row,6] = int(round(AQI_sum/count)) # average of AQIs at site
            df.iat[row,10] = 0 # Set AQS_PARAMETER_CODE to 0, since multiple
            df.iat[row,11] = '' # Set AQS_PARAMETER_DESC to empty, since multiple

            tot_count += count-1

        # Counter
        if progress % (p_mod*10) == 0:
            print('\n{0:03d}: '.format(int(progress/(p_mod*10))), end='')
        if progress % p_mod == 0:
            print('{0:04d}...'.format(int(progress/p_mod)), end='')
        progress += 1

        if row > 1: row -= 1
        else: break


    print('\n')
    end = time.time()
    elps = round(end-start)

    print('Time elapsed: {}h {}m {}s'.format(elps//3600, (elps//60)%60, elps%60))
    print(tot_count, 'rows removed')
    
    return df

In [ ]:
df_all = condense_rows(df_all, 1500) # takes 2-3 hours on my computer!

print('\nAfter condensing (no duplicates):')
df_all[df_all['Site Name'] == 'Los Angeles-North Main Street'][:5]

In [ ]:
df_all = df_all.reset_index(drop=True)
df_all = df_all.sort_values(['Site ID','Date']) # Sort again to be safe

print('Final Check:')
print(df_all.shape)
df_all[:5]

## Save to pickle file

In [ ]:
df_all.to_pickle('Data/CA2010+_dailyAQI.pkl') # save as pickle

## Append new data to file (not really sure if this works correctly)

In [ ]:
def append_new_data(*paths):
    '''
    Takes current master data and appends new data (EPA updated dataset).
    If there are datapoints retroactively added (data from a date before the
    latest date in the master data), it might not be added.
    '''
    
    df_all = pd.read_pickle('Data/CA2010+_dailyAQI.pkl')
    df_list = [pd.read_csv(path) for path in paths]
    
    df_new = pd.concat(df_list, ignore_index=True)
    df_new = df_new[df_all['Date'] > df_all.Date.max()] # only new dates
    df_new['Date'] = pd.to_datetime(df_new['Date'])
    
    df_new = remove_nan(df_new)
    df_new = condense_rows(df_new, 200*len(df_list))
    
    df_new = pd.concat((df_all, df_new))
    df_new = df_new.sort_values(['Site ID','Date'])
    df_new = df_new.reset_index()
    
    return df_new

In [ ]:
#append_new_data('Data/dailyAQI/CA2018_dailyAQI_new.csv')